# Challenge lab - AB Testing 
---

## Initial setup

To aid in resource identification later, the random and string libraries were imported and used in a helper function that generates a random string with a length of 10.
The output of the helper function was then assigned to the prefix variable.

---

In [56]:
from random import choice
from string import ascii_lowercase

def generate_random_string(length=10):
    letters = ascii_lowercase
    return ''.join(choice(letters) for i in range(length))

prefix = generate_random_string()

## Import model artifacts

To import the model artifacts, I simply specified the S3 path of the model artifacts of the Linear Learner and XGBoost models I created during the 21-day Challenge project that achieved the best results.

---

In [59]:
model_a = 's3://sagemaker-us-east-1-305262579855/sagemaker/regalado/capstone-model/linear-learner-2021-05-27-12-57-03-537/output/model.tar.gz'
model_b = 's3://sagemaker-us-east-1-305262579855/sagemaker/regalado/capstone-model/sagemaker-xgboost-2021-05-29-06-58-19-912/output/model.tar.gz'

In [60]:
import boto3

region = boto3.Session().region_name    

## Retrieve image URIs

Since two models will be used, the retrieve method was used to acquire the image URIs of the Linear Learner and XGBoost algorithms and were assigned to their corresponding variables.

---

In [61]:
from sagemaker.image_uris import retrieve

image_uri_1 = retrieve('linear-learner', region, version="1")
image_uri_2 = retrieve("xgboost", region, version="0.90-2")

## Prepare containers

In order to prepare both models for deployment, we first have to specify the containers that we will be using in our inference pipeline. 
To achieve this, we simply have to fill in the values for the Image, ContainerHostname, and ModelDataUrl attributes below.

---

In [62]:
container1 = { 
    'Image': image_uri_1,
    'ContainerHostname': 'modelA',
    'ModelDataUrl': model_a
}

container2 = { 
    'Image': image_uri_2,
    'ContainerHostname': 'modelB',
    'ModelDataUrl': model_b
}

## Init sagemaker_session and other required variables
---

In [63]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
sm_client = boto3.Session().client('sagemaker')

## Set identifiers
---

In [64]:
model_name_a = "kent-ab-model-a-" + prefix
model_name_b = "kent-ab-model-b-" + prefix
endpoint_config_name = 'kent-ab-endpoint-config-' + prefix
endpoint_name = 'kent-ab-endpoint-' + prefix

## Model creation

In this code segment, the create_model method was used to create both models in preparation for deployment. 
Next, the ModelName, ExecutionRoleArn, and the Containers attributes were filled in with the requisite variables and list of Docker containers

---

In [65]:
response = sm_client.create_model(
    ModelName        = model_name_a,
    ExecutionRoleArn = role,
    Containers       = [container1]
)

response = sm_client.create_model(
    ModelName        = model_name_b,
    ExecutionRoleArn = role,
    Containers       = [container2]
)

## Production variant description

Since we will be deploying two models for AB Testing, we need to specify a production variant description for both models. 
This is achieved using the production_variant method.

---

In [66]:
from sagemaker.session import production_variant

variant_name_a = prefix + '-variant-A'
variant_name_b = prefix + '-variant-B'

variant1 = production_variant(
    model_name=model_name_a,
    instance_type="ml.t2.medium",
    initial_instance_count=1,
    variant_name=variant_name_a,
    initial_weight=0.5
)
                              
variant2 = production_variant(
    model_name=model_name_b,
    instance_type="ml.t2.medium",
    initial_instance_count=1,
    variant_name=variant_name_b,
    initial_weight=0.5
)

## Model deployment

The models are deployed using the endpoint_from_production_variants method which creates a SageMaker endpoint from the list of production variants.

---

In [67]:
sagemaker_session.endpoint_from_production_variants(
    name=endpoint_name,
    production_variants=[variant1, variant2]
)

---------------------!

'kent-ab-endpoint-hfcmmjeflm'

## Invoke SageMaker inference endpoint

Before invoking the created inference endpoint, a low-level client representing Amazon SageMaker Runtime must first be instantiated. This is done using the client method from the AWS SDK for Python (boto3)

---

In [68]:
runtime_client = boto3.client('sagemaker-runtime')

## Prepare input data

Since both models are expecting an input data with seven feature columns, I simply copied some values from the synthetic test data from the 21-day Challenge project.
Since the invoke_endpoint method below only accepts bytes and bytearray objects, the .encode method was used to convert the string variable to a bytes object and the 
final result was stored in the sample_body variable.

---

In [153]:
temp = '0.5531491564, 1.9234457948, -0.5482002982, -1.1234940332, -0.7746149700, -2.0820987032, 0.2428820132'

sample_body = str.encode(temp)
type(sample_body)

bytes

## Invoke inference endpoint 

To invoke the endpoint, the test_endpoint helper function was used. Inside the helper function, the invoke_endpoint was used to generate the model predictions for both models

---

### variant-A: Linear Learner
### variant-B: XGBoost
---

In [154]:
from time import sleep

def test_endpoint(runtime_client, endpoint_name, body):
    response = runtime_client.invoke_endpoint(
        EndpointName = endpoint_name,
        ContentType  = 'text/csv',
        Body         = body
    )
    
    variant = response['InvokedProductionVariant']
    prediction = response['Body'].read().decode("utf-8")

    print(variant + " -> "+ prediction)
    sleep(2)

for _ in range(0, 10):
    test_endpoint(runtime_client, endpoint_name, sample_body)

hfcmmjeflm-variant-B -> 97.89613342285156
hfcmmjeflm-variant-B -> 97.89613342285156
hfcmmjeflm-variant-B -> 97.89613342285156
hfcmmjeflm-variant-B -> 97.89613342285156
hfcmmjeflm-variant-B -> 97.89613342285156
hfcmmjeflm-variant-A -> {"predictions": [{"score": -145.47348022460938}]}
hfcmmjeflm-variant-A -> {"predictions": [{"score": -145.47348022460938}]}
hfcmmjeflm-variant-A -> {"predictions": [{"score": -145.47348022460938}]}
hfcmmjeflm-variant-B -> 97.89613342285156
hfcmmjeflm-variant-A -> {"predictions": [{"score": -145.47348022460938}]}


---
### target value = 114.7073731291

In [155]:
def test_endpoint_2(runtime_client, endpoint_name, body, variant_name):
    response = runtime_client.invoke_endpoint(
        EndpointName = endpoint_name,
        ContentType  = 'text/csv',
        TargetVariant=variant_name,
        Body         = body
    )
    
    variant = response['InvokedProductionVariant']
    prediction = response['Body'].read().decode("utf-8")

    print(variant + " -> "+ prediction)
    sleep(2)

for _ in range(0, 5):
    test_endpoint_2(
        runtime_client=runtime_client, 
        endpoint_name=endpoint_name, 
        body=sample_body, 
        variant_name=variant_name_b
   )

hfcmmjeflm-variant-B -> 97.89613342285156
hfcmmjeflm-variant-B -> 97.89613342285156
hfcmmjeflm-variant-B -> 97.89613342285156
hfcmmjeflm-variant-B -> 97.89613342285156
hfcmmjeflm-variant-B -> 97.89613342285156


## Delete inference endpoint once done to avoid incurring cost
---

In [156]:
response = sm_client.delete_endpoint(EndpointName=endpoint_name)
print(f'Deleted {endpoint_name}')
response

Deleted kent-ab-endpoint-hfcmmjeflm


{'ResponseMetadata': {'RequestId': 'a5990219-e80c-4b5a-bde2-910e27987523',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a5990219-e80c-4b5a-bde2-910e27987523',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 05 Jun 2021 05:36:42 GMT'},
  'RetryAttempts': 0}}